In [4]:
%pip install google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 28.8 MB/s eta 0:00:00
  Attempting uninstall: anyio
    Found existing installation: anyio 4.4.0
    Uninstalling anyio-4.4.0:
      Successfully uninstalled anyio-4.4.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0
Note: you may need to restart the kernel to use updated packages.


In [20]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64
import os
import json
import logging
import time

In [6]:
client = genai.Client(api_key="your_api_key_here")

In [10]:
IMAGE_DIR = "./imagen3_images"

In [15]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s",
    handlers=[
        logging.FileHandler("txt2img.log", mode="a")
    ]
)

logger = logging.getLogger("Titty")

In [13]:
with open("./new_prompts.json", "r", encoding="UTF-8") as json_f:
    json_data = json.load(json_f)

In [28]:
to_skip = [filename.split(".")[0] for filename in os.listdir(IMAGE_DIR) if filename.endswith(".png")]

In [32]:
fucked_ids = []

for word_obj in json_data:
    if word_obj["id"] in to_skip:
        continue
    
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash-exp-image-generation",
            contents=f"Create this image: {word_obj['prompt']}",
            config=types.GenerateContentConfig(
              response_modalities=['Text', 'Image']
            )
        )

        for part in response.candidates[0].content.parts:
            if part.inline_data is not None:
                image = Image.open(BytesIO((part.inline_data.data)))
                image.save(f"{IMAGE_DIR}/{word_obj['id']}.png")
    except Exception as err:
        logger.error(f"[FUCKED] {word_obj!r}:: {err}")
        fucked_ids.append(word_obj["id"])

        with open("txt2img_fucked.json", "w", encoding="UTF-8") as fucked_f:
            fucked_f.write(json.dumps(fucked_ids, ensure_ascii=False, indent=4))
        continue

    time.sleep(7)
    logger.info(f"[SUCCESS] {word_obj['word']} created successfully!")